In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output
import math

df = pd.read_csv('./input/cs448b_ipasn.csv')

# data grouping

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.groupby(['date', 'l_ipn'], as_index=False).sum() # date와 l_ipn 별로 합을 구함
df['date']

In [ ]:
df['yday'] = df['date'].dt.dayofyear
df['wday'] = df['date'].dt.dayofweek

# visualization

In [ ]:
# l_ipn 기준으로 그룹 묶기
ip = []
maxF = []
for i in range(0, 10):
    ip.insert(i, df[df['l_ipn']==i])
    maxF.insert(i, np.max(ip[i]['f']))
ip[0].head()

In [ ]:
count, division = np.histogram(ip[0]['f'], bins=10)
count

In [ ]:
division

In [ ]:
# IP 별 f(날짜 별 connection 수) 분포도
f, axarray = plt.subplots(5, 2, figsize=(15, 20)) # 5x2로 10개의 flow를 보일 것이다
for i in range(10):
    count, division = np.histogram(ip[i]['f'], bins=10) # bins는 x축의 간격
    row, col = math.floor(i/2), i%2
    g = sns.barplot(x=division[0:len(division)-1], y=count, ax=axarray[row, col]) # (0, 0) 위치에 그래프 넣을 것
    axarray[row, col].set_title(f'Local IP {i} Flow')

In [ ]:
# 1년간 connection 수 흐름
f, axarray = plt.subplots(5, 2, figsize=(15, 20))
for i in range(10):
    row, col = math.floor(i/2), i%2
    axarray[row, col].plot(ip[i]['yday'], ip[i]['f'])
    axarray[row, col].plot(ip[i]['yday'], [ip[i]['f'].mean() + 3*ip[i]['f'].std()]*len(ip[i]['yday']), color='g')

In [ ]:
ip[0] = df[df['l_ipn']==0]
maxF[0] = np.max(ip[0]['f'])
ip[1] = df[df['l_ipn']==1][0:len(ip[1]['f'])-5]
maxF[1] = np.max(ip[1]['f'])
ip[2] = df[df['l_ipn']==2]
maxF[2] = np.max(ip[2]['f'])
ip[3] = df[df['l_ipn']==3]
maxF[3] = np.max(ip[3]['f'])
ip[4] = df[df['l_ipn']==4][0:len(ip[4]['f'])-7]